Step 1: Import the necessary libraries

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

Load Data

In [4]:
def load_imdb_data(data_file):
    df = pd.read_csv(data_file)
    texts = df['OM_Regular'].tolist()
    labels = [1 if sentiment == "P" else 0 for sentiment in df['OM_Prediction'].tolist()]
    return texts, labels

In [5]:
data_file = "9_OM_dynamic_40p_60np.csv"
texts, labels = load_imdb_data(data_file)

Step 3: Create a custom dataset class for text classification

In [6]:
class TextClassificationDataset(Dataset):
      def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
      def __len__(self):
        return len(self.texts)
      def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

Step 4: Build our custom BERT classifier

In [7]:
class BERTClassifier(nn.Module):
  def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

  def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

Step 5: Define the train() function

In [8]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

Step 6: Build our evaluation method


In [9]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

Step 7: Build our prediction method

In [10]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        return "P" if preds.item() == 1 else "NP"

Step 8: Define our model’s parameters

In [11]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

Step 9: Loading and splitting the data.

In [12]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


Step 10: Initialize tokenizer, dataset, and data loader

In [13]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Step 11: Set up the device and model

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Step 12: Set up optimizer and learning rate scheduler

In [15]:
optimizer = AdamW(model.parameters(), lr=learning_rate, no_deprecation_warning=True)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

Step 13: Training the model

In [16]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/4
Validation Accuracy: 0.8139
              precision    recall  f1-score   support

           0       0.98      0.70      0.82      1571
           1       0.69      0.98      0.81      1083

    accuracy                           0.81      2654
   macro avg       0.84      0.84      0.81      2654
weighted avg       0.86      0.81      0.81      2654

Epoch 2/4
Validation Accuracy: 0.8199
              precision    recall  f1-score   support

           0       0.98      0.71      0.82      1571
           1       0.70      0.98      0.82      1083

    accuracy                           0.82      2654
   macro avg       0.84      0.84      0.82      2654
weighted avg       0.87      0.82      0.82      2654

Epoch 3/4
Validation Accuracy: 0.8199
              precision    recall  f1-score   support

           0       0.98      0.71      0.82      1571
           1       0.70      0.98      0.82      1083

    accuracy                           0.82      2654
   macro avg  

Saving the trained model

In [17]:
torch.save(model.state_dict(), "bert_classifier_om_dataset.pth")

Step 14: Evaluating our model’s performance

In [20]:
# ORM status prediction
test_text = "module OM_name one sig class1_name extends Class attrSet = c1_at1+c1_at2 attrSet = c1_at1+c1_at2 isAbstract = No no parent one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type one sig class2_name extends Class id=Integer isAbstract = No no parent one sig c2_at1 extends c2_at1_type one sig c2_at1 extends c2_at1_type one sig assoc1 extends Association src = class1_name dst = class2_name src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc one sig class3_name extends Class attrSet = c3_at1 one parent parent in class1_name isAbstract = No attrSet = c1_at1+c1_at2 one sig c3_at1 extends c3_at1_type pred show run show for 16, OM_name_Solution : 0 Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class1_name Attribute c1_at2: c1_at2_type Table: class2_name Attribute c2_at2: c1_at2_type Table: class3_name Attribute c3_at1: c3_at1_type Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class3_name Attribute c1_at1: c1_at1_type Foreign Key Table: class3_name Parent Class class1_name Table Name: class1_name Table Name: class2_name Table Name: class3_name Association Table assoc1 Source : class1_name Association Table assoc1 Destination : class2_name One To Many Association Source > Destination Mapping Strategy of Table class1_name: map_str2 Mapping Strategy of Table class3_name: map_str3 Association Mapping Strategy: Foreign Key Embedding, USE OM_name_0 Table structure for table class2_name CREATE TABLE `class2_name` ( `c2_at2` c2_at2_type `c2_at1` c2_at1_type NOT NULL `c1_at1` c1_at1_type, KEY `FK_class2_name_c1_at1_idx` (`c1_at1`) PRIMARY KEY (`orderID`) Table structure for table class3_name CREATE TABLE `class3_name` `c3_at1` int `c1_at1` c1_at1_type NOT NULL KEY `FK_class3_name_c1_at1_idx` (`c1_at1`) PRIMARY KEY (`c1_at1`) Table structure for table class1_name CREATE TABLE `class1_name` `c1_at2` c1_at2_type `c1_at1` c1_at1_type NOT NULL PRIMARY KEY (`c1_at1`) ALTER TABLE `class2_name` ADD CONSTRAINT `FK_class2_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE; ALTER TABLE `class3_name` ADD CONSTRAINT `FK_class3_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE"
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("module OM_name one sig class1_name extends Class attrSet = c1_at1+c1_at2 attrSet = c1_at1+c1_at2 isAbstract = No no parent one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type one sig class2_name extends Class id=Integer isAbstract = No no parent one sig c2_at1 extends c2_at1_type one sig c2_at1 extends c2_at1_type one sig assoc1 extends Association src = class1_name dst = class2_name src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc one sig class3_name extends Class attrSet = c3_at1 one parent parent in class1_name isAbstract = No attrSet = c1_at1+c1_at2 one sig c3_at1 extends c3_at1_type pred show run show for 16, OM_name_Solution : 0 Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class1_name Attribute c1_at2: c1_at2_type Table: class2_name Attribute c2_at2: c1_at2_type Table: class3_name Attribute c3_at1: c3_at1_type Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class3_name Attribute c1_at1: c1_at1_type Foreign Key Table: class3_name Parent Class class1_name Table Name: class1_name Table Name: class2_name Table Name: class3_name Association Table assoc1 Source : class1_name Association Table assoc1 Destination : class2_name One To Many Association Source > Destination Mapping Strategy of Table class1_name: map_str2 Mapping Strategy of Table class3_name: map_str3 Association Mapping Strategy: Foreign Key Embedding, USE OM_name_0 Table structure for table class2_name CREATE TABLE `class2_name` ( `c2_at2` c2_at2_type `c2_at1` c2_at1_type NOT NULL `c1_at1` c1_at1_type, KEY `FK_class2_name_c1_at1_idx` (`c1_at1`) PRIMARY KEY (`orderID`) Table structure for table class3_name CREATE TABLE `class3_name` `c3_at1` int `c1_at1` c1_at1_type NOT NULL KEY `FK_class3_name_c1_at1_idx` (`c1_at1`) PRIMARY KEY (`c1_at1`) Table structure for table class1_name CREATE TABLE `class1_name` `c1_at2` c1_at2_type `c1_at1` c1_at1_type NOT NULL PRIMARY KEY (`c1_at1`) ALTER TABLE `class2_name` ADD CONSTRAINT `FK_class2_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE; ALTER TABLE `class3_name` ADD CONSTRAINT `FK_class3_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE")
print(f"status: {sentiment}")

module OM_name one sig class1_name extends Class attrSet = c1_at1+c1_at2 attrSet = c1_at1+c1_at2 isAbstract = No no parent one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type one sig class2_name extends Class id=Integer isAbstract = No no parent one sig c2_at1 extends c2_at1_type one sig c2_at1 extends c2_at1_type one sig assoc1 extends Association src = class1_name dst = class2_name src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc one sig class3_name extends Class attrSet = c3_at1 one parent parent in class1_name isAbstract = No attrSet = c1_at1+c1_at2 one sig c3_at1 extends c3_at1_type pred show run show for 16, OM_name_Solution : 0 Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class1_name Attribute c1_at2: c1_at2_type Table: class2_name Attribute c2_at2: c1_at2_type Table: class3_name Attribute c3_at1: c3_at1_type Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class3_name Attribute c1_at1: c1_at1_type Foreign Key Ta

In [21]:
# ORM status prediction
test_text = "module OM_name one sig class1_name extends Class attrSet = c1_at1+c1_at2 attrSet = c1_at1+c1_at2 isAbstract = No no parent one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type one sig class2_name extends Class id=Integer isAbstract = No no parent one sig c2_at1 extends c2_at1_type one sig c2_at1 extends c2_at1_type one sig assoc1 extends Association src = class1_name dst = class2_name src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc one sig class3_name extends Class attrSet = c3_at1 one parent parent in class1_name isAbstract = No attrSet = c1_at1+c1_at2 one sig c3_at1 extends c3_at1_type pred show run show for 16, OM_name_Solution : 1 Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class1_name Attribute c1_at2: c1_at2_type Table: class2_name Attribute c2_at2: c1_at2_type Table: class3_name Attribute c3_at1: c3_at1_type Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class3_name Attribute c1_at1: c1_at1_type Foreign Key Table: class3_name Parent Class class1_name Table Name: class1_name Table Name: class2_name Table Name: class3_name Association Table assoc1 Source : class1_name Association Table assoc1 Destination : class2_name One To Many Association Source > Destination Mapping Strategy of Table class1_name: map_str2 Mapping Strategy of Table class2_name: map_str2 Mapping Strategy of Table class3_name: map_str2 Association Mapping Strategy: Own Association Table, USE OM_name_0 Table structure for table class2_name CREATE TABLE `class2_name` ( `c2_at2` c2_at2_type `c2_at1` c2_at1_type NOT NULL `c1_at1` c1_at1_type, KEY `FK_class2_name_c1_at1_idx` (`c1_at1`) PRIMARY KEY (`orderID`) Table structure for table class1_name CREATE TABLE `class1_name` `c1_at3` c1_at3_type(64) `c1_at2` c1_at2_type `c3_at1` int `c1_at1` c1_at1_type NOT NULL PRIMARY KEY (`c1_at1`) ALTER TABLE `class2_name` ADD CONSTRAINT `FK_class2_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE"
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("module OM_name one sig class1_name extends Class attrSet = c1_at1+c1_at2 attrSet = c1_at1+c1_at2 isAbstract = No no parent one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type one sig class2_name extends Class id=Integer isAbstract = No no parent one sig c2_at1 extends c2_at1_type one sig c2_at1 extends c2_at1_type one sig assoc1 extends Association src = class1_name dst = class2_name src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc one sig class3_name extends Class attrSet = c3_at1 one parent parent in class1_name isAbstract = No attrSet = c1_at1+c1_at2 one sig c3_at1 extends c3_at1_type pred show run show for 16, OM_name_Solution : 1 Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class1_name Attribute c1_at2: c1_at2_type Table: class2_name Attribute c2_at2: c1_at2_type Table: class3_name Attribute c3_at1: c3_at1_type Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class3_name Attribute c1_at1: c1_at1_type Foreign Key Table: class3_name Parent Class class1_name Table Name: class1_name Table Name: class2_name Table Name: class3_name Association Table assoc1 Source : class1_name Association Table assoc1 Destination : class2_name One To Many Association Source > Destination Mapping Strategy of Table class1_name: map_str2 Mapping Strategy of Table class2_name: map_str2 Mapping Strategy of Table class3_name: map_str2 Association Mapping Strategy: Own Association Table, USE OM_name_0 Table structure for table class2_name CREATE TABLE `class2_name` ( `c2_at2` c2_at2_type `c2_at1` c2_at1_type NOT NULL `c1_at1` c1_at1_type, KEY `FK_class2_name_c1_at1_idx` (`c1_at1`) PRIMARY KEY (`orderID`) Table structure for table class1_name CREATE TABLE `class1_name` `c1_at3` c1_at3_type(64) `c1_at2` c1_at2_type `c3_at1` int `c1_at1` c1_at1_type NOT NULL PRIMARY KEY (`c1_at1`) ALTER TABLE `class2_name` ADD CONSTRAINT `FK_class2_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE")
print(f"status: {sentiment}")

module OM_name one sig class1_name extends Class attrSet = c1_at1+c1_at2 attrSet = c1_at1+c1_at2 isAbstract = No no parent one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type one sig class2_name extends Class id=Integer isAbstract = No no parent one sig c2_at1 extends c2_at1_type one sig c2_at1 extends c2_at1_type one sig assoc1 extends Association src = class1_name dst = class2_name src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc one sig class3_name extends Class attrSet = c3_at1 one parent parent in class1_name isAbstract = No attrSet = c1_at1+c1_at2 one sig c3_at1 extends c3_at1_type pred show run show for 16, OM_name_Solution : 1 Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class1_name Attribute c1_at2: c1_at2_type Table: class2_name Attribute c2_at2: c1_at2_type Table: class3_name Attribute c3_at1: c3_at1_type Table: class1_name Attribute c1_at1: c1_at1_type Primary Key Table: class3_name Attribute c1_at1: c1_at1_type Foreign Key Ta

In [22]:
# ORM status prediction
test_text = "module OM_name: 0, open Declaration one sig class1_name extends Class attrSet = c1_at1+c1_at2 id=c1_at1 no parent isAbstract = No } one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type, one sig class2_name extends Class attrSet = c2_at1+c2_at2 id = c2_at1 no parent isAbstract = No } one sig c2_at1 extends c2_at1_type one sig c2_at2 extends c2_at2_type, one sig class3_name extends Class attrSet = c3_at1+c3_at2+c3_at3+c3_at4+c3_at5+c3_at6 id=c3_at1 no parent isAbstract = No } one sig c3_at1 extends c3_at1_type one sig c3_at2 extends c3_at2_type one sig c3_at3 extends c3_at3_type one sig c3_at4 extends c3_at4_type one sig c3_at5 extends c3_at5_type one sig c3_at6 extends c3_at6_type, one sig class4_name extends Class attrSet = c4_at1+c4_at2+c4_at3+c4_at4 id=c4_at3 no parent isAbstract = No } one sig c4_at1 extends c4_at1_type one sig c4_at2 extends c4_at2_type one sig c4_at3 extends c4_at3_type one sig c4_at4 extends c4_at4_type, one sig class5_name extends Class attrSet = c5_at1 one parent parent in class2_name id=c2_at1 isAbstract = No } one sig c5_at1 extends c5_at1_type, one sig class6_name extends Class attrSet = c6_at1_0+c6_at2+c6_at3+c6_at4 id=c6_at2 no parent isAbstract = No } one sig c6_at1_0 extends c6_at1_type one sig c6_at2 extends c6_at2_type one sig c6_at3 extends c6_at3_type one sig c6_at4 extends c6_at4_type, one sig class7_name extends Class attrSet = c7_at1 one parent parent in class2_name id=c2_at1 isAbstract = No } one sig c7_at1 extends c7_at1_type, one sig class8_name extends Class attrSet = c8_at1+c8_at2+c2_at2+c6_at3 id=c8_at1 no parent isAbstract = No } one sig c8_at1 extends c8_at1_type one sig c8_at1 extends c8_at1_type, one sig assoc1 extends Association src = class8_name dst = class2_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc2 extends Association src = class2_name dst = class4_name, src_multiplicity = src_mlpc2 dst_multiplicity = dst_mlpc, } one sig assoc3 extends Association src = class8_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc4 extends Association src = class8_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc5 extends Association src = class8_name dst = class6_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2, } one sig assoc6 extends Association src = class6_name dst = class1_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2, } one sig assoc7 extends Association src = class6_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc8 extends Association src = class1_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc9 extends Association src = class1_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc10 extends Association src = class2_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2,},Mapping Strategy for class1_name : map_str2Mapping Strategy for class5_name : map_str2Mapping Strategy for class7_name : map_str2Mapping Strategy for class8_name : map_str2Association Strategy for assoc4 : assoc_str1Association Strategy for assoc7 : assoc_str1Association Strategy for assoc8 : assoc_str1Association Strategy for assoc1 : assoc_str2Association Strategy for assoc10 : assoc_str2Association Strategy for assoc3 : assoc_str2Association Strategy for assoc5 : assoc_str2Association Strategy for assoc6 : assoc_str2Association Strategy for assoc9 : assoc_str2,USE OM_name:0;CREATE TABLE `class3_name` (`c3_at6` c3_at6_type,`c3_at5` c3_at5_type,`c3_at4` c3_at4_type,`c3_at3` c3_at3_type,`c3_at2` c3_at2_type,`c3_at1` c3_at1_type NOT NULL,PRIMARY KEY (`c3_at1`)CREATE TABLE `class1_name` (`c1_at2` c1_at2_type(64),`c1_at1` c1_at1_type NOT NULL,PRIMARY KEY (`c1_at1`)CREATE TABLE `class2_name` (`c8_at1` c8_at1_type,`c2_at1` c2_at1_type NOT NULL,KEY `FK_class2_name_c8_at1_idx` (`c8_at1`),PRIMARY KEY (`c2_at1`)CREATE TABLE `class7_name` (`c7_at1` c7_at1_type(64),`c2_at1` c2_at1_type NOT NULL,KEY `FK_class7_name_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c2_at1`)CREATE TABLE `assoc10` (`c3_at1` c3_at1_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc10_c3_at1_idx` (`c3_at1`),KEY `FK_assoc10_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c3_at1`,`c2_at1`)CREATE TABLE `class8_name` (`c8_at2` c8_at2_type(64),`c6_at3` c6_at3_type(64),`c2_at2` c2_at2_type(64),`c8_at1` c8_at1_type NOT NULL,PRIMARY KEY (`c8_at1`)CREATE TABLE `class5_name` (`c5_at1` c5_at1_type,`c2_at1` c2_at1_type NOT NULL,KEY `FK_class5_name_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c2_at1`)Association Strategy for assoc5 : assoc_str2`c8_at1` c8_at1_type NOT NULL,`c6_at2` c6_at2_type NOT NULL,KEY `FK_assoc5_c8_at1_idx` (`c8_at1`),KEY `FK_assoc5_c6_at2_idx` (`c6_at2`),PRIMARY KEY (`c8_at1`,`c6_at2`)CREATE TABLE `assoc3` (`c8_at1` c8_at1_type NOT NULL,`c3_at1` c3_at1_type NOT NULL,KEY `FK_assoc3_c8_at1_idx` (`c8_at1`),KEY `FK_assoc3_c3_at1_idx` (`c3_at1`),PRIMARY KEY (`c8_at1`,`c3_at1`)CREATE TABLE `assoc8` (`c3_at1` c3_at1_type NOT NULL,`c1_at1` c1_at1_type NOT NULL,KEY `FK_assoc8_c3_at1_idx` (`c3_at1`),KEY `FK_assoc8_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c3_at1`,`c1_at1`)CREATE TABLE `class4_name` (`c4_at4` c4_at4_type(64),`c4_at2` c4_at2_type(64),`c4_at1` c4_at1_type(64),`c6_at2` c6_at2_type,`c4_at3` c4_at3_type NOT NULL,KEY `FK_class4_name_c6_at2_idx` (`c6_at2`),PRIMARY KEY (`c4_at3`)CREATE TABLE `assoc4` (`c8_at1` c8_at1_type NOT NULL,`c4_at3` c4_at3_type NOT NULL,KEY `FK_assoc4_c8_at1_idx` (`c8_at1`),KEY `FK_assoc4_c4_at3_idx` (`c4_at3`),PRIMARY KEY (`c8_at1`,`c4_at3`)CREATE TABLE `assoc6` (`c6_at2` c6_at2_type NOT NULL,`c1_at1` c1_at1_type NOT NULL,KEY `FK_assoc6_c6_at2_idx` (`c6_at2`),KEY `FK_assoc6_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c6_at2`,`c1_at1`)CREATE TABLE `class6_name` (`c6_at4` c6_at4_type(64),`c6_at1_0` c6_at1_type(64),`c6_at2` c6_at2_type NOT NULL,PRIMARY KEY (`c6_at2`)CREATE TABLE `assoc9` (`c4_at3` c4_at3_type NOT NULL,`c1_at1` c1_at1_type NOT NULL,KEY `FK_assoc9_c4_at3_idx` (`c4_at3`),KEY `FK_assoc9_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c4_at3`,`c1_at1`)CREATE TABLE `assoc2` (`c4_at3` c4_at3_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc2_c4_at3_idx` (`c4_at3`),KEY `FK_assoc2_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c4_at3`,`c2_at1`)ALTER TABLE `class2_name`ADD CONSTRAINT `FK_class2_name_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class7_name`ADD CONSTRAINT `FK_class7_name_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc10`ADD CONSTRAINT `FK_assoc10_c3_at1` FOREIGN KEY (`c3_at1`) REFERENCES `class3_name` (`c3_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc10_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class5_name`ADD CONSTRAINT `FK_class5_name_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc5`ADD CONSTRAINT `FK_assoc5_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc5_c6_at2` FOREIGN KEY (`c6_at2`) REFERENCES `class6_name` (`c6_at2`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc3`ADD CONSTRAINT `FK_assoc3_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc3_c3_at1` FOREIGN KEY (`c3_at1`) REFERENCES `class3_name` (`c3_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc8`ADD CONSTRAINT `FK_assoc8_c3_at1` FOREIGN KEY (`c3_at1`) REFERENCES `class3_name` (`c3_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc8_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class4_name`ADD CONSTRAINT `FK_class4_name_c6_at2` FOREIGN KEY (`c6_at2`) REFERENCES `class6_name` (`c6_at2`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc4`ADD CONSTRAINT `FK_assoc4_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc4_c4_at3` FOREIGN KEY (`c4_at3`) REFERENCES `class4_name` (`c4_at3`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc6`ALTER TABLE `class7_name`ADD CONSTRAINT `FK_assoc6_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc9`ADD CONSTRAINT `FK_assoc9_c4_at3` FOREIGN KEY (`c4_at3`) REFERENCES `class4_name` (`c4_at3`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc9_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc2`ADD CONSTRAINT `FK_assoc2_c4_at3` FOREIGN KEY (`c4_at3`) REFERENCES `class4_name` (`c4_at3`) ON DELETE CASCADE ON UPDATE CASCADEADD CONSTRAINT `FK_assoc2_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE"
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("module OM_name: 0, open Declaration one sig class1_name extends Class attrSet = c1_at1+c1_at2 id=c1_at1 no parent isAbstract = No } one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type, one sig class2_name extends Class attrSet = c2_at1+c2_at2 id = c2_at1 no parent isAbstract = No } one sig c2_at1 extends c2_at1_type one sig c2_at2 extends c2_at2_type, one sig class3_name extends Class attrSet = c3_at1+c3_at2+c3_at3+c3_at4+c3_at5+c3_at6 id=c3_at1 no parent isAbstract = No } one sig c3_at1 extends c3_at1_type one sig c3_at2 extends c3_at2_type one sig c3_at3 extends c3_at3_type one sig c3_at4 extends c3_at4_type one sig c3_at5 extends c3_at5_type one sig c3_at6 extends c3_at6_type, one sig class4_name extends Class attrSet = c4_at1+c4_at2+c4_at3+c4_at4 id=c4_at3 no parent isAbstract = No } one sig c4_at1 extends c4_at1_type one sig c4_at2 extends c4_at2_type one sig c4_at3 extends c4_at3_type one sig c4_at4 extends c4_at4_type, one sig class5_name extends Class attrSet = c5_at1 one parent parent in class2_name id=c2_at1 isAbstract = No } one sig c5_at1 extends c5_at1_type, one sig class6_name extends Class attrSet = c6_at1_0+c6_at2+c6_at3+c6_at4 id=c6_at2 no parent isAbstract = No } one sig c6_at1_0 extends c6_at1_type one sig c6_at2 extends c6_at2_type one sig c6_at3 extends c6_at3_type one sig c6_at4 extends c6_at4_type, one sig class7_name extends Class attrSet = c7_at1 one parent parent in class2_name id=c2_at1 isAbstract = No } one sig c7_at1 extends c7_at1_type, one sig class8_name extends Class attrSet = c8_at1+c8_at2+c2_at2+c6_at3 id=c8_at1 no parent isAbstract = No } one sig c8_at1 extends c8_at1_type one sig c8_at1 extends c8_at1_type, one sig assoc1 extends Association src = class8_name dst = class2_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc2 extends Association src = class2_name dst = class4_name, src_multiplicity = src_mlpc2 dst_multiplicity = dst_mlpc, } one sig assoc3 extends Association src = class8_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc4 extends Association src = class8_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc5 extends Association src = class8_name dst = class6_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2, } one sig assoc6 extends Association src = class6_name dst = class1_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2, } one sig assoc7 extends Association src = class6_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc8 extends Association src = class1_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc9 extends Association src = class1_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc10 extends Association src = class2_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2,},Mapping Strategy for class1_name : map_str2Mapping Strategy for class5_name : map_str2Mapping Strategy for class7_name : map_str2Mapping Strategy for class8_name : map_str2Association Strategy for assoc4 : assoc_str1Association Strategy for assoc7 : assoc_str1Association Strategy for assoc8 : assoc_str1Association Strategy for assoc1 : assoc_str2Association Strategy for assoc10 : assoc_str2Association Strategy for assoc3 : assoc_str2Association Strategy for assoc5 : assoc_str2Association Strategy for assoc6 : assoc_str2Association Strategy for assoc9 : assoc_str2,USE OM_name:0;CREATE TABLE `class3_name` (`c3_at6` c3_at6_type,`c3_at5` c3_at5_type,`c3_at4` c3_at4_type,`c3_at3` c3_at3_type,`c3_at2` c3_at2_type,`c3_at1` c3_at1_type NOT NULL,PRIMARY KEY (`c3_at1`)CREATE TABLE `class1_name` (`c1_at2` c1_at2_type(64),`c1_at1` c1_at1_type NOT NULL,PRIMARY KEY (`c1_at1`)CREATE TABLE `class2_name` (`c8_at1` c8_at1_type,`c2_at1` c2_at1_type NOT NULL,KEY `FK_class2_name_c8_at1_idx` (`c8_at1`),PRIMARY KEY (`c2_at1`)CREATE TABLE `class7_name` (`c7_at1` c7_at1_type(64),`c2_at1` c2_at1_type NOT NULL,KEY `FK_class7_name_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c2_at1`)CREATE TABLE `assoc10` (`c3_at1` c3_at1_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc10_c3_at1_idx` (`c3_at1`),KEY `FK_assoc10_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c3_at1`,`c2_at1`)CREATE TABLE `class8_name` (`c8_at2` c8_at2_type(64),`c6_at3` c6_at3_type(64),`c2_at2` c2_at2_type(64),`c8_at1` c8_at1_type NOT NULL,PRIMARY KEY (`c8_at1`)CREATE TABLE `class5_name` (`c5_at1` c5_at1_type,`c2_at1` c2_at1_type NOT NULL,KEY `FK_class5_name_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c2_at1`)Association Strategy for assoc5 : assoc_str2`c8_at1` c8_at1_type NOT NULL,`c6_at2` c6_at2_type NOT NULL,KEY `FK_assoc5_c8_at1_idx` (`c8_at1`),KEY `FK_assoc5_c6_at2_idx` (`c6_at2`),PRIMARY KEY (`c8_at1`,`c6_at2`)CREATE TABLE `assoc3` (`c8_at1` c8_at1_type NOT NULL,`c3_at1` c3_at1_type NOT NULL,KEY `FK_assoc3_c8_at1_idx` (`c8_at1`),KEY `FK_assoc3_c3_at1_idx` (`c3_at1`),PRIMARY KEY (`c8_at1`,`c3_at1`)CREATE TABLE `assoc8` (`c3_at1` c3_at1_type NOT NULL,`c1_at1` c1_at1_type NOT NULL,KEY `FK_assoc8_c3_at1_idx` (`c3_at1`),KEY `FK_assoc8_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c3_at1`,`c1_at1`)CREATE TABLE `class4_name` (`c4_at4` c4_at4_type(64),`c4_at2` c4_at2_type(64),`c4_at1` c4_at1_type(64),`c6_at2` c6_at2_type,`c4_at3` c4_at3_type NOT NULL,KEY `FK_class4_name_c6_at2_idx` (`c6_at2`),PRIMARY KEY (`c4_at3`)CREATE TABLE `assoc4` (`c8_at1` c8_at1_type NOT NULL,`c4_at3` c4_at3_type NOT NULL,KEY `FK_assoc4_c8_at1_idx` (`c8_at1`),KEY `FK_assoc4_c4_at3_idx` (`c4_at3`),PRIMARY KEY (`c8_at1`,`c4_at3`)CREATE TABLE `assoc6` (`c6_at2` c6_at2_type NOT NULL,`c1_at1` c1_at1_type NOT NULL,KEY `FK_assoc6_c6_at2_idx` (`c6_at2`),KEY `FK_assoc6_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c6_at2`,`c1_at1`)CREATE TABLE `class6_name` (`c6_at4` c6_at4_type(64),`c6_at1_0` c6_at1_type(64),`c6_at2` c6_at2_type NOT NULL,PRIMARY KEY (`c6_at2`)CREATE TABLE `assoc9` (`c4_at3` c4_at3_type NOT NULL,`c1_at1` c1_at1_type NOT NULL,KEY `FK_assoc9_c4_at3_idx` (`c4_at3`),KEY `FK_assoc9_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c4_at3`,`c1_at1`)CREATE TABLE `assoc2` (`c4_at3` c4_at3_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc2_c4_at3_idx` (`c4_at3`),KEY `FK_assoc2_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c4_at3`,`c2_at1`)ALTER TABLE `class2_name`ADD CONSTRAINT `FK_class2_name_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class7_name`ADD CONSTRAINT `FK_class7_name_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc10`ADD CONSTRAINT `FK_assoc10_c3_at1` FOREIGN KEY (`c3_at1`) REFERENCES `class3_name` (`c3_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc10_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class5_name`ADD CONSTRAINT `FK_class5_name_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc5`ADD CONSTRAINT `FK_assoc5_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc5_c6_at2` FOREIGN KEY (`c6_at2`) REFERENCES `class6_name` (`c6_at2`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc3`ADD CONSTRAINT `FK_assoc3_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc3_c3_at1` FOREIGN KEY (`c3_at1`) REFERENCES `class3_name` (`c3_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc8`ADD CONSTRAINT `FK_assoc8_c3_at1` FOREIGN KEY (`c3_at1`) REFERENCES `class3_name` (`c3_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc8_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class4_name`ADD CONSTRAINT `FK_class4_name_c6_at2` FOREIGN KEY (`c6_at2`) REFERENCES `class6_name` (`c6_at2`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc4`ADD CONSTRAINT `FK_assoc4_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc4_c4_at3` FOREIGN KEY (`c4_at3`) REFERENCES `class4_name` (`c4_at3`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc6`ALTER TABLE `class7_name`ADD CONSTRAINT `FK_assoc6_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc9`ADD CONSTRAINT `FK_assoc9_c4_at3` FOREIGN KEY (`c4_at3`) REFERENCES `class4_name` (`c4_at3`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc9_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc2`ADD CONSTRAINT `FK_assoc2_c4_at3` FOREIGN KEY (`c4_at3`) REFERENCES `class4_name` (`c4_at3`) ON DELETE CASCADE ON UPDATE CASCADEADD CONSTRAINT `FK_assoc2_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE")
print(f"status: {sentiment}")

module OM_name: 0, open Declaration one sig class1_name extends Class attrSet = c1_at1+c1_at2 id=c1_at1 no parent isAbstract = No } one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type, one sig class2_name extends Class attrSet = c2_at1+c2_at2 id = c2_at1 no parent isAbstract = No } one sig c2_at1 extends c2_at1_type one sig c2_at2 extends c2_at2_type, one sig class3_name extends Class attrSet = c3_at1+c3_at2+c3_at3+c3_at4+c3_at5+c3_at6 id=c3_at1 no parent isAbstract = No } one sig c3_at1 extends c3_at1_type one sig c3_at2 extends c3_at2_type one sig c3_at3 extends c3_at3_type one sig c3_at4 extends c3_at4_type one sig c3_at5 extends c3_at5_type one sig c3_at6 extends c3_at6_type, one sig class4_name extends Class attrSet = c4_at1+c4_at2+c4_at3+c4_at4 id=c4_at3 no parent isAbstract = No } one sig c4_at1 extends c4_at1_type one sig c4_at2 extends c4_at2_type one sig c4_at3 extends c4_at3_type one sig c4_at4 extends c4_at4_type, one sig class5_name extends Class attrSet 

In [23]:
# ORM status prediction
test_text = "module OM_name: 0, open Declaration one sig class1_name extends Class attrSet = c1_at1+c1_at2 id=c1_at1 no parent isAbstract = No } one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type, one sig class2_name extends Class attrSet = c2_at1+c2_at2 id = c2_at1 no parent isAbstract = No } one sig c2_at1 extends c2_at1_type one sig c2_at2 extends c2_at2_type, one sig class3_name extends Class attrSet = c3_at1+c3_at2+c3_at3+c3_at4+c3_at5+c3_at6 id=c3_at1 no parent isAbstract = No } one sig c3_at1 extends c3_at1_type one sig c3_at2 extends c3_at2_type one sig c3_at3 extends c3_at3_type one sig c3_at4 extends c3_at4_type one sig c3_at5 extends c3_at5_type one sig c3_at6 extends c3_at6_type, one sig class4_name extends Class attrSet = c4_at1+c4_at2+c4_at3+c4_at4 id=c4_at3 no parent isAbstract = No } one sig c4_at1 extends c4_at1_type one sig c4_at2 extends c4_at2_type one sig c4_at3 extends c4_at3_type one sig c4_at4 extends c4_at4_type, one sig class5_name extends Class attrSet = c5_at1 one parent parent in class2_name id=c2_at1 isAbstract = No } one sig c5_at1 extends c5_at1_type, one sig class6_name extends Class attrSet = c6_at1_0+c6_at2+c6_at3+c6_at4 id=c6_at2 no parent isAbstract = No } one sig c6_at1_0 extends c6_at1_type one sig c6_at2 extends c6_at2_type one sig c6_at3 extends c6_at3_type one sig c6_at4 extends c6_at4_type, one sig class7_name extends Class attrSet = c7_at1 one parent parent in class2_name id=c2_at1 isAbstract = No } one sig c7_at1 extends c7_at1_type, one sig class8_name extends Class attrSet = c8_at1+c8_at2+c2_at2+c6_at3 id=c8_at1 no parent isAbstract = No } one sig c8_at1 extends c8_at1_type one sig c8_at1 extends c8_at1_type, one sig assoc1 extends Association src = class8_name dst = class2_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc2 extends Association src = class2_name dst = class4_name, src_multiplicity = src_mlpc2 dst_multiplicity = dst_mlpc, } one sig assoc3 extends Association src = class8_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc4 extends Association src = class8_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc5 extends Association src = class8_name dst = class6_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2, } one sig assoc6 extends Association src = class6_name dst = class1_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2, } one sig assoc7 extends Association src = class6_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc8 extends Association src = class1_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc9 extends Association src = class1_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc10 extends Association src = class2_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2,},Mapping Strategy for class1_name : map_str2Mapping Strategy for class2_name : map_str2Mapping Strategy for class5_name : map_str2Mapping Strategy for class6_name : map_str2Mapping Strategy for class7_name : map_str2Association Strategy for assoc3 : assoc_str1Association Strategy for assoc1 : assoc_str2Association Strategy for assoc10 : assoc_str2Association Strategy for assoc4 : assoc_str2Association Strategy for assoc5 : assoc_str2Association Strategy for assoc6 : assoc_str2Association Strategy for assoc7 : assoc_str2Association Strategy for assoc8 : assoc_str2Association Strategy for assoc9 : assoc_str2,USE OM_name:0;CREATE TABLE `class3_name` (`c8_at1` c8_at1_type,`c3_at6` c3_at6_type,`c3_at5` c3_at5_type,`c3_at4` c3_at4_type,`c3_at3` c3_at3_type,`c3_at2` c3_at2_type,`c3_at1` c3_at1_type NOT NULL,`c1_at1` int,KEY `FK_class3_name_c8_at1_idx` (`c8_at1`),KEY `FK_class3_name_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c3_at1`)CREATE TABLE `class1_name` (`c1_at2` c1_at2_type(64),`c1_at1` c1_at1_type NOT NULL,PRIMARY KEY (`c1_at1`)CREATE TABLE `class2_name` (`c2_at2` c2_at2_type(64),`c2_at1` c2_at1_type NOT NULL,PRIMARY KEY (`c2_at1`)CREATE TABLE `class7_name` (`c7_at1` c7_at1_type(64),`c2_at2` c2_at2_type(64),`c2_at1` c2_at1_type NOT NULL,PRIMARY KEY (`c2_at1`)CREATE TABLE `assoc10` (`c3_at1` c3_at1_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc10_c3_at1_idx` (`c3_at1`),KEY `FK_assoc10_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c3_at1`,`c2_at1`)CREATE TABLE `class8_name` (`c8_at2` c8_at2_type(64),`c2_at2` c2_at2_type(64),`c8_at1` c8_at1_type NOT NULL,PRIMARY KEY (`c8_at1`)CREATE TABLE `class5_name` (`c5_at1` c5_at1_type,`c2_at1` c2_at1_type NOT NULL,KEY `FK_class5_name_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c2_at1`)Association Strategy for assoc5 : assoc_str2`c8_at1` c8_at1_type NOT NULL,`c6_at2` c6_at2_type NOT NULL,KEY `FK_assoc5_c8_at1_idx` (`c8_at1`),KEY `FK_assoc5_c6_at2_idx` (`c6_at2`),PRIMARY KEY (`c8_at1`,`c6_at2`)CREATE TABLE `assoc1` (`c8_at1` c8_at1_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc1_c8_at1_idx` (`c8_at1`),KEY `FK_assoc1_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c8_at1`,`c2_at1`)CREATE TABLE `class4_name` (`c4_at4` c4_at4_type(64),`c4_at2` c4_at2_type(64),`c4_at1` c4_at1_type(64),`c8_at1` c8_at1_type,`c6_at2` c6_at2_type,`c4_at3` c4_at3_type NOT NULL,`c1_at1` int,KEY `FK_class4_name_c8_at1_idx` (`c8_at1`),KEY `FK_class4_name_c6_at2_idx` (`c6_at2`),KEY `FK_class4_name_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c4_at3`)CREATE TABLE `assoc6` (`c6_at2` c6_at2_type NOT NULL,`c1_at1` c1_at1_type NOT NULL,KEY `FK_assoc6_c6_at2_idx` (`c6_at2`),KEY `FK_assoc6_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c6_at2`,`c1_at1`)CREATE TABLE `class6_name` (`c6_at4` c6_at4_type(64),`c6_at3` c6_at3_type(64),`c6_at1_0` c6_at1_type(64),`c6_at2` c6_at2_type NOT NULL,PRIMARY KEY (`c6_at2`)CREATE TABLE `assoc2` (`c4_at3` c4_at3_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc2_c4_at3_idx` (`c4_at3`),KEY `FK_assoc2_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c4_at3`,`c2_at1`)ALTER TABLE `class3_name`ADD CONSTRAINT `FK_class3_name_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_class3_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc10`ADD CONSTRAINT `FK_assoc10_c3_at1` FOREIGN KEY (`c3_at1`) REFERENCES `class3_name` (`c3_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc10_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class5_name`ADD CONSTRAINT `FK_class5_name_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc5`ADD CONSTRAINT `FK_assoc5_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc5_c6_at2` FOREIGN KEY (`c6_at2`) REFERENCES `class6_name` (`c6_at2`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc1`ADD CONSTRAINT `FK_assoc1_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class4_name`ADD CONSTRAINT `FK_class4_name_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_class4_name_c6_at2` FOREIGN KEY (`c6_at2`) REFERENCES `class6_name` (`c6_at2`) ON DELETE CASCADE ON UPDATE CASCADE;ADD CONSTRAINT `FK_class4_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc6`ALTER TABLE `class7_name`ADD CONSTRAINT `FK_assoc6_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc2`ADD CONSTRAINT `FK_assoc2_c4_at3` FOREIGN KEY (`c4_at3`) REFERENCES `class4_name` (`c4_at3`) ON DELETE CASCADE ON UPDATE CASCADEADD CONSTRAINT `FK_assoc2_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE"
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print("module OM_name: 0, open Declaration one sig class1_name extends Class attrSet = c1_at1+c1_at2 id=c1_at1 no parent isAbstract = No } one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type, one sig class2_name extends Class attrSet = c2_at1+c2_at2 id = c2_at1 no parent isAbstract = No } one sig c2_at1 extends c2_at1_type one sig c2_at2 extends c2_at2_type, one sig class3_name extends Class attrSet = c3_at1+c3_at2+c3_at3+c3_at4+c3_at5+c3_at6 id=c3_at1 no parent isAbstract = No } one sig c3_at1 extends c3_at1_type one sig c3_at2 extends c3_at2_type one sig c3_at3 extends c3_at3_type one sig c3_at4 extends c3_at4_type one sig c3_at5 extends c3_at5_type one sig c3_at6 extends c3_at6_type, one sig class4_name extends Class attrSet = c4_at1+c4_at2+c4_at3+c4_at4 id=c4_at3 no parent isAbstract = No } one sig c4_at1 extends c4_at1_type one sig c4_at2 extends c4_at2_type one sig c4_at3 extends c4_at3_type one sig c4_at4 extends c4_at4_type, one sig class5_name extends Class attrSet = c5_at1 one parent parent in class2_name id=c2_at1 isAbstract = No } one sig c5_at1 extends c5_at1_type, one sig class6_name extends Class attrSet = c6_at1_0+c6_at2+c6_at3+c6_at4 id=c6_at2 no parent isAbstract = No } one sig c6_at1_0 extends c6_at1_type one sig c6_at2 extends c6_at2_type one sig c6_at3 extends c6_at3_type one sig c6_at4 extends c6_at4_type, one sig class7_name extends Class attrSet = c7_at1 one parent parent in class2_name id=c2_at1 isAbstract = No } one sig c7_at1 extends c7_at1_type, one sig class8_name extends Class attrSet = c8_at1+c8_at2+c2_at2+c6_at3 id=c8_at1 no parent isAbstract = No } one sig c8_at1 extends c8_at1_type one sig c8_at1 extends c8_at1_type, one sig assoc1 extends Association src = class8_name dst = class2_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc2 extends Association src = class2_name dst = class4_name, src_multiplicity = src_mlpc2 dst_multiplicity = dst_mlpc, } one sig assoc3 extends Association src = class8_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc4 extends Association src = class8_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc5 extends Association src = class8_name dst = class6_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2, } one sig assoc6 extends Association src = class6_name dst = class1_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2, } one sig assoc7 extends Association src = class6_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc8 extends Association src = class1_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc9 extends Association src = class1_name dst = class4_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc, } one sig assoc10 extends Association src = class2_name dst = class3_name, src_multiplicity = src_mlpc dst_multiplicity = dst_mlpc2,},Mapping Strategy for class1_name : map_str2Mapping Strategy for class2_name : map_str2Mapping Strategy for class5_name : map_str2Mapping Strategy for class6_name : map_str2Mapping Strategy for class7_name : map_str2Association Strategy for assoc3 : assoc_str1Association Strategy for assoc1 : assoc_str2Association Strategy for assoc10 : assoc_str2Association Strategy for assoc4 : assoc_str2Association Strategy for assoc5 : assoc_str2Association Strategy for assoc6 : assoc_str2Association Strategy for assoc7 : assoc_str2Association Strategy for assoc8 : assoc_str2Association Strategy for assoc9 : assoc_str2,USE OM_name:0;CREATE TABLE `class3_name` (`c8_at1` c8_at1_type,`c3_at6` c3_at6_type,`c3_at5` c3_at5_type,`c3_at4` c3_at4_type,`c3_at3` c3_at3_type,`c3_at2` c3_at2_type,`c3_at1` c3_at1_type NOT NULL,`c1_at1` int,KEY `FK_class3_name_c8_at1_idx` (`c8_at1`),KEY `FK_class3_name_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c3_at1`)CREATE TABLE `class1_name` (`c1_at2` c1_at2_type(64),`c1_at1` c1_at1_type NOT NULL,PRIMARY KEY (`c1_at1`)CREATE TABLE `class2_name` (`c2_at2` c2_at2_type(64),`c2_at1` c2_at1_type NOT NULL,PRIMARY KEY (`c2_at1`)CREATE TABLE `class7_name` (`c7_at1` c7_at1_type(64),`c2_at2` c2_at2_type(64),`c2_at1` c2_at1_type NOT NULL,PRIMARY KEY (`c2_at1`)CREATE TABLE `assoc10` (`c3_at1` c3_at1_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc10_c3_at1_idx` (`c3_at1`),KEY `FK_assoc10_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c3_at1`,`c2_at1`)CREATE TABLE `class8_name` (`c8_at2` c8_at2_type(64),`c2_at2` c2_at2_type(64),`c8_at1` c8_at1_type NOT NULL,PRIMARY KEY (`c8_at1`)CREATE TABLE `class5_name` (`c5_at1` c5_at1_type,`c2_at1` c2_at1_type NOT NULL,KEY `FK_class5_name_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c2_at1`)Association Strategy for assoc5 : assoc_str2`c8_at1` c8_at1_type NOT NULL,`c6_at2` c6_at2_type NOT NULL,KEY `FK_assoc5_c8_at1_idx` (`c8_at1`),KEY `FK_assoc5_c6_at2_idx` (`c6_at2`),PRIMARY KEY (`c8_at1`,`c6_at2`)CREATE TABLE `assoc1` (`c8_at1` c8_at1_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc1_c8_at1_idx` (`c8_at1`),KEY `FK_assoc1_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c8_at1`,`c2_at1`)CREATE TABLE `class4_name` (`c4_at4` c4_at4_type(64),`c4_at2` c4_at2_type(64),`c4_at1` c4_at1_type(64),`c8_at1` c8_at1_type,`c6_at2` c6_at2_type,`c4_at3` c4_at3_type NOT NULL,`c1_at1` int,KEY `FK_class4_name_c8_at1_idx` (`c8_at1`),KEY `FK_class4_name_c6_at2_idx` (`c6_at2`),KEY `FK_class4_name_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c4_at3`)CREATE TABLE `assoc6` (`c6_at2` c6_at2_type NOT NULL,`c1_at1` c1_at1_type NOT NULL,KEY `FK_assoc6_c6_at2_idx` (`c6_at2`),KEY `FK_assoc6_c1_at1_idx` (`c1_at1`),PRIMARY KEY (`c6_at2`,`c1_at1`)CREATE TABLE `class6_name` (`c6_at4` c6_at4_type(64),`c6_at3` c6_at3_type(64),`c6_at1_0` c6_at1_type(64),`c6_at2` c6_at2_type NOT NULL,PRIMARY KEY (`c6_at2`)CREATE TABLE `assoc2` (`c4_at3` c4_at3_type NOT NULL,`c2_at1` c2_at1_type NOT NULL,KEY `FK_assoc2_c4_at3_idx` (`c4_at3`),KEY `FK_assoc2_c2_at1_idx` (`c2_at1`),PRIMARY KEY (`c4_at3`,`c2_at1`)ALTER TABLE `class3_name`ADD CONSTRAINT `FK_class3_name_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_class3_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc10`ADD CONSTRAINT `FK_assoc10_c3_at1` FOREIGN KEY (`c3_at1`) REFERENCES `class3_name` (`c3_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc10_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class5_name`ADD CONSTRAINT `FK_class5_name_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc5`ADD CONSTRAINT `FK_assoc5_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_assoc5_c6_at2` FOREIGN KEY (`c6_at2`) REFERENCES `class6_name` (`c6_at2`) ON DELETE CASCADE ON UPDATE CASCADE;ALTER TABLE `assoc1`ADD CONSTRAINT `FK_assoc1_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `class4_name`ADD CONSTRAINT `FK_class4_name_c8_at1` FOREIGN KEY (`c8_at1`) REFERENCES `class8_name` (`c8_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ADD CONSTRAINT `FK_class4_name_c6_at2` FOREIGN KEY (`c6_at2`) REFERENCES `class6_name` (`c6_at2`) ON DELETE CASCADE ON UPDATE CASCADE;ADD CONSTRAINT `FK_class4_name_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc6`ALTER TABLE `class7_name`ADD CONSTRAINT `FK_assoc6_c1_at1` FOREIGN KEY (`c1_at1`) REFERENCES `class1_name` (`c1_at1`) ON DELETE CASCADE ON UPDATE CASCADE,ALTER TABLE `assoc2`ADD CONSTRAINT `FK_assoc2_c4_at3` FOREIGN KEY (`c4_at3`) REFERENCES `class4_name` (`c4_at3`) ON DELETE CASCADE ON UPDATE CASCADEADD CONSTRAINT `FK_assoc2_c2_at1` FOREIGN KEY (`c2_at1`) REFERENCES `class2_name` (`c2_at1`) ON DELETE CASCADE ON UPDATE CASCADE")
print(f"status: {sentiment}")

module OM_name: 0, open Declaration one sig class1_name extends Class attrSet = c1_at1+c1_at2 id=c1_at1 no parent isAbstract = No } one sig c1_at1 extends c1_at1_type one sig c1_at2 extends c1_at2_type, one sig class2_name extends Class attrSet = c2_at1+c2_at2 id = c2_at1 no parent isAbstract = No } one sig c2_at1 extends c2_at1_type one sig c2_at2 extends c2_at2_type, one sig class3_name extends Class attrSet = c3_at1+c3_at2+c3_at3+c3_at4+c3_at5+c3_at6 id=c3_at1 no parent isAbstract = No } one sig c3_at1 extends c3_at1_type one sig c3_at2 extends c3_at2_type one sig c3_at3 extends c3_at3_type one sig c3_at4 extends c3_at4_type one sig c3_at5 extends c3_at5_type one sig c3_at6 extends c3_at6_type, one sig class4_name extends Class attrSet = c4_at1+c4_at2+c4_at3+c4_at4 id=c4_at3 no parent isAbstract = No } one sig c4_at1 extends c4_at1_type one sig c4_at2 extends c4_at2_type one sig c4_at3 extends c4_at3_type one sig c4_at4 extends c4_at4_type, one sig class5_name extends Class attrSet 